<a id='toc'></a>
# New Features from Bureau Data
In this notebook we will create new features from the *bureau* and *bureau_balance* data. Recall that *bureau* contains information about credit extended by other financial institutions that has been reported to a credit bureau. *bureau_balance* contains repayment history information on credits in *bureau*.
1. [Setting up the Environment](#envir)
1. [Read in Initial Processed Data](#read)
1. [A Quick Look at the Dataframes](#look)
1. [Creating a New Feature from *bureau_balance*](#b_bal)
1. [Creating New Features from *bureau*](#bur)
     1. [Aggregating the Numerical Attributes](#bur_num)
     1. [Counting Categorical Levels and Calculating Proportions](#bur_cat)
1. [Joining All New Features with *app*](#join)


<a id='envir'></a>
# 1. Setting up the Environment

[Return](#toc)

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from numpy import nan as NaN
#import os
#import warnings
#warnings.filterwarnings('ignore')
pd.options.display.max_rows = 150
pd.options.display.max_columns = 200

<a id='read'></a>
# 2. Read in Initial Preprocessed Data

[Return](#toc)

In [2]:
# Read in the initial preprocessed data
path = 'C:/Users/Richard/Documents/A_Documents/Cap_Data/CSV'
app = pd.read_csv(path + '/initial_output/app_init.csv')
bureau = pd.read_csv(path + '/initial_output/bureau_init.csv')
bureau_balance = pd.read_csv(path + '/initial_output/bureau_bal_init.csv')

<a id='look'></a>
# 3. A Quick Look at the Dataframes

[Return](#toc)

In [3]:
# A quick check and glimpse at the data
print('app shape: '+str(app.shape))
print('bureau shape: '+str(bureau.shape))
print('bureau_balance shape: '+str(bureau_balance.shape))

app shape: (307511, 99)
bureau shape: (1716428, 12)
bureau_balance shape: (27299925, 3)


In [4]:
app.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_FLAG
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,0,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,1,0,0,0.0,0.0,1.0,False
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188.0,-1186.0,-291,NaN,1,0,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,1,0,0,0.0,0.0,0.0,False
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225.0,-4260.0,-2531,26.0,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0.0,0.0,0.0,False
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039.0,-9833.0,-2437,NaN,1,0,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,Business Entity Type 3,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,CREDIT_TYPE,DAYS_CREDIT_UPDATE
0,215354,5714462,Closed,currency 1,-497,-153.0,NaN,91323.0,0.0,NaN,Consumer credit,-131.0
1,215354,5714463,Active,currency 1,-208,NaN,NaN,225000.0,171342.0,NaN,Credit card,-20.0
2,215354,5714464,Active,currency 1,-203,NaN,NaN,464323.5,NaN,NaN,Consumer credit,-16.0
3,215354,5714465,Active,currency 1,-203,NaN,NaN,90000.0,NaN,NaN,Credit card,-16.0
4,215354,5714466,Active,currency 1,-629,NaN,77674.5,2700000.0,NaN,NaN,Consumer credit,-21.0


In [6]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


<a id='b_bal'></a>
# 4. Creating a New Feature from *bureau_balance*
For each credit in *bureau* there is (where available) a time series in *bureau_balance* containing details of the repayment history of the applicant for that credit. The information is a simple flag indicating, at that time, the status of the credit: C = closed, X = unknown, 0 = no payment overdue, 1 = payment 0-30 days overdue, 2 = payment 31-60 days over due, ..., 5 = 120+ overdue, sold or written off.

We reduce each time series to a single flag based on a hierarchical labelling scheme starting with the 'worst' flag down to the 'best' flag:
1. First look for a '5'; if the series contains a '5' then it is labelled as '5'
2. Next look for a '4';  if the series contains a '4' then it is labelled as '4'

and so on down to 'X'

[Return](#toc)

In [7]:
# A look at the unique values in the STATUS attribute
print(bureau_balance['STATUS'].unique())
bureau_balance['STATUS'].unique().dtype


['C' '0' 'X' '1' '2' '3' '5' '4']


dtype('O')

In [8]:
# A label function to classify each time series
def label(x):
    '''
    A function to classify the time series in bureau_balance.
    The function takes a pandas series and returns the first encounted value
    from 5, 4, 3, 2, 1, 0, C, X
    '''
    if '5' in x.values:
        return '5'
    else:
        if '4' in x.values:
            return '4'
        else:
            if '3' in x.values:
                return '3'
            else:
                if '2' in x.values:
                    return '2'
                else:
                    if '1' in x.values:
                        return '1'
                    else:
                        if '0' in x.values:
                            return '0'
                        else:
                            if 'C' in x.values:
                                return 'C'
                            else:
                                if 'X' in x.values:
                                    return 'X'
                                else:
                                    return "ERROR" 

In [9]:
# Run the labelling scheme and create a new dataframe to hold the results
%time bureau_bal_label = bureau_balance.groupby('SK_ID_BUREAU', as_index=False)\
.agg({'STATUS': label})

Wall time: 23.6 s


In [10]:
# A look at the new labelled data
bureau_bal_label.head()

,SK_ID_BUREAU,STATUS
0,5001709,C
1,5001710,0
2,5001711,0
3,5001712,0
4,5001713,X


The new dataframe *bureau_bal_label* will now be joined with the dataframe *bureau* on the credit ID ('SK_ID_BUREAU') but first let's examine the relationship between credit IDs in *bureau* and *bureau_bal_label*.

In [11]:
# Checking the relationship between credit IDs in bureau and bureau_bal_label
# The number of unique credit IDs in bureau_bal_label
print('Unique credit IDs in bureau_bal_label: ' + str(bureau_bal_label['SK_ID_BUREAU'].nunique()))
# The number of unique credit IDs in bureau
print('Unique credit IDs in bureau: ' + str(bureau['SK_ID_BUREAU'].nunique()))

Unique credit IDs in bureau_bal_label: 817395
Unique credit IDs in bureau: 1716428


Not every credit in *bureau* has a historical payment label in *bureau_bal_label*. As such, there will be some NaN values in our new attribute following the join. We will replace the NaNs with 'NoHist' showing that there is no repayment history available.

In [12]:
# Checking to see is every credit ID in bureau_bal_label exists in bureau?
sum(np.isin(bureau_bal_label['SK_ID_BUREAU'].unique(),bureau['SK_ID_BUREAU']))

774354

The above number is less than the number of credit IDs in *bureau_bal_label*. Therefore, there are some credit IDs in *bureau_bal_label* that do not exist in *bureau*. Unfortunately, these observations are of no value to us because they will not have a corresponding applicant in our main *app* dataframe. (I suspect these credit IDs relate to applicants in the Kaggle test file we are not using.)

In [13]:
# Joining the new feature to bureau
bureau = pd.merge(bureau, bureau_bal_label, on='SK_ID_BUREAU', how='left')

In [14]:
# Replacing NaNs with 'NoHist'
bureau['STATUS'].fillna('NoHist', inplace=True)

In [15]:
# A look at the new dataframe. The new attribute 'STATUS' is on the far right.
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,CREDIT_TYPE,DAYS_CREDIT_UPDATE,STATUS
0,215354,5714462,Closed,currency 1,-497,-153.0,NaN,91323.0,0.0,NaN,Consumer credit,-131.0,NoHist
1,215354,5714463,Active,currency 1,-208,NaN,NaN,225000.0,171342.0,NaN,Credit card,-20.0,NoHist
2,215354,5714464,Active,currency 1,-203,NaN,NaN,464323.5,NaN,NaN,Consumer credit,-16.0,NoHist
3,215354,5714465,Active,currency 1,-203,NaN,NaN,90000.0,NaN,NaN,Credit card,-16.0,NoHist
4,215354,5714466,Active,currency 1,-629,NaN,77674.5,2700000.0,NaN,NaN,Consumer credit,-21.0,NoHist


<a id='bur'></a>
# 5. Creating New Features from *bureau*
All numeric attributes will be aggregated at the applicant ID level with a count, maximum, minimum and mean value. 

All categorical attributes will be aggregated at the applicant ID level by a count and 'proportion' for each level of the categorical attribute.
<a id='bur_num'></a>

[Return](#toc)
## A. Aggregating the Numerical Attributes

[Return](#toc)

In [16]:
# Inspired by code on Kaggle from Will Koehrsen
# Remove the 'SK_ID_BUREAU' as it's no longer needed
bureau.drop('SK_ID_BUREAU', axis=1, inplace=True)

# Aggregating the numeric attributes
bureau_agg = bureau.groupby('SK_ID_CURR', as_index = False).agg(['count', 'max', 'min', 'mean']).reset_index()

# A look at the aggregated values
bureau_agg.head(10)

SK_ID_CURR DAYS_CREDIT                          DAYS_ENDDATE_FACT          \
                   count   max   min         mean             count     max   
0     100001           7   -49 -1572  -735.000000                 4  -544.0   
1     100002           8  -103 -1437  -874.000000                 6   -36.0   
2     100003           4  -606 -2586 -1400.750000                 3  -540.0   
3     100004           2  -408 -1326  -867.000000                 2  -382.0   
4     100005           3   -62  -373  -190.666667                 1  -123.0   
5     100007           1 -1149 -1149 -1149.000000                 1  -783.0   
6     100008           3   -78 -1097  -757.333333                 2  -790.0   
7     100009          18  -239 -2882 -1271.500000                14  -313.0   
8     100010           2 -1138 -2741 -1939.500000                 1 -1138.0   
9     100011           4 -1309 -2508 -1773.000000                 4  -968.0   

                       AMT_CREDIT_MAX_OVERDUE                            \
      min         mean                  count        max  min      mean   
0 -1328.0  -825.500000                      0        NaN  NaN       NaN   
1 -1185.0  -697.500000                      5   5043.645  0.0  1681.029   
2 -2131.0 -1097.333333                      4      0.000  0.0     0.000   
3  -683.0  -532.500000                      1      0.000  0.0     0.000   
4  -123.0  -123.000000                      1      0.000  0.0     0.000   
5  -783.0  -783.000000                      1      0.000  0.0     0.000   
6 -1028.0  -909.000000                      1      0.000  0.0     0.000   
7 -2152.0 -1108.500000                      4      0.000  0.0     0.000   
8 -1138.0 -1138.000000                      0        NaN  NaN       NaN   
9 -2197.0 -1463.250000                      2  10147.230  0.0  5073.615   

  AMT_CREDIT_SUM                                     AMT_CREDIT_SUM_DEBT  \
           count        max       min           mean               count   
0              7   378000.0   85500.0  207623.571429                   7   
1              8   450000.0       0.0  108131.945625                   5   
2              4   810000.0   22248.0  254350.125000                   4   
3              2    94537.8   94500.0   94518.900000                   2   
4              3   568800.0   29826.0  219042.000000                   3   
5              1   146250.0  146250.0  146250.000000                   1   
6              3   267606.0   95134.5  156148.500000                   3   
7             18  1777500.0   35770.5  266711.750000                  14   
8              2   675000.0  315000.0  495000.000000                   2   
9              4   145242.0   54000.0  108807.075000                   3   

                                AMT_CREDIT_SUM_LIMIT                   \
        max  min           mean                count         max  min   
0  373239.0  0.0   85240.928571                    6       0.000  0.0   
1  245781.0  0.0   49156.200000                    4   31988.565  0.0   
2       0.0  0.0       0.000000                    4  810000.000  0.0   
3       0.0  0.0       0.000000                    2       0.000  0.0   
4  543087.0  0.0  189469.500000                    3       0.000  0.0   
5       0.0  0.0       0.000000                    1       0.000  0.0   
6  240057.0  0.0   80019.000000                    3       0.000  0.0   
7  557959.5  0.0   76953.535714                   11       0.000  0.0   
8  348007.5  0.0  174003.750000                    1       0.000  0.0   
9       0.0  0.0       0.000000                    3       0.000  0.0   

                DAYS_CREDIT_UPDATE                              
           mean              count    max     min         mean  
0       0.00000                  7   -6.0  -155.0   -93.142857  
1    7997.14125                  8   -7.0 -1185.0  -499.875000  
2  202500.00000                  4  -43.0 -2131.0  -816.000000  
3       0.00000                  2 -3

In [17]:
# Inspired by code on Kaggle from Will Koehrsen
# Creating new 'single level' column names

# New list to hold new column names
col=['SK_ID_CURR']

# Iterating through original column names
for lev_zero in bureau_agg.columns.levels[0][:-1]:
    
    #Iterating through new aggregated features
    for lev_one in bureau_agg.columns.levels[1][:-1]:
        
        col.append('bureau' + '_' + lev_zero + '_' + lev_one)

# Replacing old column names with new names
bureau_agg.columns = col

In [18]:
# Look at new column names
bureau_agg.head(10)

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_mean,bureau_DAYS_ENDDATE_FACT_count,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_AMT_CREDIT_MAX_OVERDUE_count,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_SUM_count,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_count,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_DEBT_min,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_count,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_LIMIT_min,bureau_AMT_CREDIT_SUM_LIMIT_mean,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean
0,100001,7,-49,-1572,-735.000000,4,-544.0,-1328.0,-825.500000,0,NaN,NaN,NaN,7,378000.0,85500.0,207623.571429,7,373239.0,0.0,85240.928571,6,0.000,0.0,0.00000,7,-6.0,-155.0,-93.142857
1,100002,8,-103,-1437,-874.000000,6,-36.0,-1185.0,-697.500000,5,5043.645,0.0,1681.029,8,450000.0,0.0,108131.945625,5,245781.0,0.0,49156.200000,4,31988.565,0.0,7997.14125,8,-7.0,-1185.0,-499.875000
2,100003,4,-606,-2586,-1400.750000,3,-540.0,-2131.0,-1097.333333,4,0.000,0.0,0.000,4,810000.0,22248.0,254350.125000,4,0.0,0.0,0.000000,4,810000.000,0.0,202500.00000,4,-43.0,-2131.0,-816.000000
3,100004,2,-408,-1326,-867.000000,2,-382.0,-683.0,-532.500000,1,0.000,0.0,0.000,2,94537.8,94500.0,94518.900000,2,0.0,0.0,0.000000,2,0.000,0.0,0.00000,2,-382.0,-682.0,-532.000000
4,100005,3,-62,-373,-190.666667,1,-123.0,-123.0,-123.000000,1,0.000,0.0,0.000,3,568800.0,29826.0,219042.000000,3,543087.0,0.0,189469.500000,3,0.000,0.0,0.00000,3,-11.0,-121.0,-54.333333
5,100007,1,-1149,-1149,-1149.000000,1,-783.0,-783.0,-783.000000,1,0.000,0.0,0.000,1,146250.0,146250.0,146250.000000,1,0.0,0.0,0.000000,1,0.000,0.0,0.00000,1,-783.0,-783.0,-783.000000
6,100008,3,-78,-1097,-757.333333,2,-790.0,-1028.0,-909.000000,1,0.000,0.0,0.000,3,267606.0,95134.5,156148.500000,3,240057.0,0.0,80019.000000,3,0.000,0.0,0.00000,3,-16.0,-1027.0,-611.000000
7,100009,18,-239,-2882,-1271.500000,14,-313.0,-2152.0,-1108.500000,4,0.000,0.0,0.000,18,1777500.0,35770.5,266711.750000,14,557959.5,0.0,76953.535714,11,0.000,0.0,0.00000,18,-23.0,-2152.0,-851.611111
8,100010,2,-1138,-2741,-1939.500000,1,-1138.0,-1138.0,-1138.000000,0,NaN,NaN,NaN,2,675000.0,315000.0,495000.000000,2,348007.5,0.0,174003.750000,1,0.000,0.0,0.00000,2,-18.0,-1138.0,-578.000000
9,100011,4,-1309,-2508,-1773.000000,4,-968.0,-2197.0,-1463.250000,2,10147.230,0.0,5073.615,4,145242.0,54000.0,108807.075000,3,0.0,0.0,0.000000,3,0.000,0.0,0.00000,4,-965.0,-2172.0,-1454.750000


<a id='bur_cat'></a>
## B. Counting Categorical Levels and Calculating Proportions

[Return](#toc)

In [19]:
# Inspired by code on Kaggle from Will Koehrsen
# Encoding the categorical attributes
cat_freq = pd.get_dummies(bureau.select_dtypes('object'))

# Adding back the applicant ID column
cat_freq['SK_ID_CURR'] = bureau['SK_ID_CURR']

# Calculating the count for each level (using 'sum') and the proportion of that level (using 'mean')
cat_freq_agg = cat_freq.groupby(['SK_ID_CURR'], as_index=False).agg(['sum', 'mean']).reset_index()

# A look at the aggregated data
cat_freq_agg.head()

SK_ID_CURR CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Bad debt       \
                              sum      mean                    sum mean   
0     100001                    3  0.428571                      0  0.0   
1     100002                    2  0.250000                      0  0.0   
2     100003                    1  0.250000                      0  0.0   
3     100004                    0  0.000000                      0  0.0   
4     100005                    2  0.666667                      0  0.0   

  CREDIT_ACTIVE_Closed           CREDIT_ACTIVE_Sold       \
                   sum      mean                sum mean   
0                    4  0.571429                  0  0.0   
1                    6  0.750000                  0  0.0   
2                    3  0.750000                  0  0.0   
3                    2  1.000000                  0  0.0   
4                    1  0.333333                  0  0.0   

  CREDIT_CURRENCY_currency 1      CREDIT_CURRENCY_currency 2       \
                         sum mean                        sum mean   
0                          7  1.0                          0  0.0   
1                          8  1.0                          0  0.0   
2                          4  1.0                          0  0.0   
3                          2  1.0                          0  0.0   
4                          3  1.0                          0  0.0   

  CREDIT_CURRENCY_currency 3      CREDIT_CURRENCY_currency 4       \
                         sum mean                        sum mean   
0                          0  0.0                          0  0.0   
1                          0  0.0                          0  0.0   
2                          0  0.0                          0  0.0   
3                          0  0.0                          0  0.0   
4                          0  0.0                          0  0.0   

  CREDIT_TYPE_Another type of loan      CREDIT_TYPE_Car loan       \
                               sum mean                  sum mean   
0                                0  0.0                    0  0.0   
1                                0  0.0                    0  0.0   
2                                0  0.0                    0  0.0   
3                                0  0.0                    0  0.0   
4                                0  0.0                    0  0.0   

  CREDIT_TYPE_Cash loan (non-earmarked)      CREDIT_TYPE_Consumer credit  \
                                    sum mean                         sum   
0                                     0  0.0                           7   
1                                     0  0.0                           4   
2                                     0  0.0                           2   
3                                     0  0.0                           2   
4                                     0  0.0                           2   

            CREDIT_TYPE_Credit card           CREDIT_TYPE_Interbank credit  \
       mean                     sum      mean                          sum   
0  1.000000                       0  0.000000                            0   
1  0.500000                       4  0.500000                            0   
2  0.500000                       2  0.500000                            0   
3  1.000000                       0  0.000000                            0   
4  0.666667                       1  0.333333                            0   

       CREDIT_TYPE_Loan for business development       \
  mean                                       sum mean   
0  0.0                                         0  0.0   
1  0.0                                         0  0.0   
2  0.0                                         0  0.0   
3  0.0                                         0  0.0   
4  0.0                                         0  0.0   

  CREDIT_TYPE_Loan for purchase of shares (margin lending)       \
                                                       s

In [20]:
# Inspired by code on Kaggle from Will Koehrsen
# Creating new 'single level' column names

# List to hold new column names
col = ['SK_ID_CURR']

# Iterating through original column names
for lev_zero in cat_freq_agg.columns.levels[0][:-1]:
    col.append('bureau' + '_' + lev_zero + '_' + 'count')
    col.append('bureau' + '_' + lev_zero + '_' + 'fraction')

# Replacing old column names with new names
cat_freq_agg.columns = col 

# A look at the new column names
cat_freq_agg.head()

,SK_ID_CURR,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_fraction,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_fraction,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_fraction,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_fraction,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_fraction,bureau_CREDIT_CURRENCY_currency 2_count,bureau_CREDIT_CURRENCY_currency 2_fraction,bureau_CREDIT_CURRENCY_currency 3_count,bureau_CREDIT_CURRENCY_currency 3_fraction,bureau_CREDIT_CURRENCY_currency 4_count,bureau_CREDIT_CURRENCY_currency 4_fraction,bureau_CREDIT_TYPE_Another type of loan_count,bureau_CREDIT_TYPE_Another type of loan_fraction,bureau_CREDIT_TYPE_Car loan_count,bureau_CREDIT_TYPE_Car loan_fraction,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_fraction,bureau_CREDIT_TYPE_Consumer credit_count,bureau_CREDIT_TYPE_Consumer credit_fraction,bureau_CREDIT_TYPE_Credit card_count,bureau_CREDIT_TYPE_Credit card_fraction,bureau_CREDIT_TYPE_Interbank credit_count,bureau_CREDIT_TYPE_Interbank credit_fraction,bureau_CREDIT_TYPE_Loan for business development_count,bureau_CREDIT_TYPE_Loan for business development_fraction,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_fraction,bureau_CREDIT_TYPE_Loan for the purchase of equipment_count,bureau_CREDIT_TYPE_Loan for the purchase of equipment_fraction,bureau_CREDIT_TYPE_Loan for working capital replenishment_count,bureau_CREDIT_TYPE_Loan for working capital replenishment_fraction,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_fraction,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_fraction,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_fraction,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_fraction,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_fraction,bureau_STATUS_0_count,bureau_STATUS_0_fraction,bureau_STATUS_1_count,bureau_STATUS_1_fraction,bureau_STATUS_2_count,bureau_STATUS_2_fraction,bureau_STATUS_3_count,bureau_STATUS_3_fraction,bureau_STATUS_4_count,bureau_STATUS_4_fraction,bureau_STATUS_5_count,bureau_STATUS_5_fraction,bureau_STATUS_C_count,bureau_STATUS_C_fraction,bureau_STATUS_NoHist_count,bureau_STATUS_NoHist_fraction,bureau_STATUS_X_count,bureau_STATUS_X_fraction
0,100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,7,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,6,0.857143,1,0.142857,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,4,0.500000,4,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,0.250000,6,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,0.500000,2,0.500000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,4,1.0,0,0.0
3,100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,1.0,0,0.0
4,100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2,0.666667,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,3,1.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


<a id='join'></a>
# 6. Joining All New Features with *app*
We will now join all the newly created features with the *app* dataframe on the application ID ('SK_ID_CURR').

[Return](#toc)

In [21]:
# Joining the numerical features
app = pd.merge(app, bureau_agg, on='SK_ID_CURR', how='left')

In [22]:
# Joining the categorical features
app = pd.merge(app, cat_freq_agg, on='SK_ID_CURR', how='left')

In [23]:
# A look at the updated 'app' dataframe
app.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_FLAG,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_mean,bureau_DAYS_ENDDATE_FACT_count,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_AMT_CREDIT_MAX_OVERDUE_count,bureau_AMT_CREDIT_MAX_OVERDUE_max,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_SUM_count,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_count,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_DEBT_min,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_LIMIT_count,bureau_AMT_CREDIT_SUM_LIMIT_max,bureau_AMT_CREDIT_SUM_LIMIT_min,bureau_AMT_CREDIT_SUM_LIMIT_mean,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_fraction,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_fraction,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_fraction,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_fraction,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_fraction,bureau_CREDIT_CURRENCY_currency 2_count,bureau_CREDIT_CURRENCY_currency 2_fraction,bureau_CREDIT_CURRENCY_currency 3_count,bureau_CREDIT_CURRENCY_currency 3_fraction,bureau_CREDIT_CURRENCY_currency 4_count,bureau_CREDIT_CURRENCY_currency 4_fraction,bureau_CREDIT_TYPE_Another type of loan_count,bureau_CREDIT_TYPE_Another type of loan_fraction,bureau_CREDIT_TYPE_Car loan_count,bureau_CREDIT_TYPE_Car loan_fraction,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count,bureau_CREDIT_TYPE_Cash loan (non-earmarked)_fraction,bureau_CREDIT_TYPE_Consumer credit_count,bureau_CREDIT_TYPE_Consumer credit_fraction,bureau_CREDIT_TYPE_Credit card_count,bureau_CREDIT_TYPE_Credit card_fraction,bureau_CREDIT_TYPE_Interbank credit_count,bureau_CREDIT_TYPE_Interbank credit_fraction,bureau_CREDIT_TYPE_Loan for business development_count,bureau_CREDIT_TYPE_Loan for business development_fraction,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_fr

In [24]:
# Saving the new 'app' dataframe to CSV file 
path = path+'/app_with_features/'
app.to_csv(path+'app_features.csv', index=False)

In [25]:
sum(app['bureau_CREDIT_TYPE_Microloan_count'].isnull())

44020